In [40]:
#libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, euclidean, hamming
from sklearn.preprocessing import normalize

In [41]:
#read the data & have randome sample for reproducability
recipes = pd.read_csv('../data/full_clean_data.csv')
recipes.sample(3, random_state=123)

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed
7358,Maple Dill Carrots,https://www.allrecipes.com/recipe/87694/maple-...,side-dish,4.57,729,"['3 cups peeled and sliced carrots ', ' 2 tabl...",20,4,4 servings,117.3,16.1,6.0,1.0,"['carrot', 'butter', 'sugar']"
2656,Dill Pickle Soup,https://www.allrecipes.com/recipe/54804/dill-p...,soups-stews-and-chili,4.31,205,"['2 tablespoons butter ', ' ½ cup all-purpose ...",30,8,8 servings,104.3,13.7,4.2,3.1,"['butter', 'flour', 'chicken broth', 'pickle',..."
12521,White Bean Tabbouleh,https://www.allrecipes.com/recipe/205611/white...,salad,4.74,46,"['1 cup bulgur wheat, uncooked ', ' 1\u2009½ c...",110,6,6 servings,197.2,24.8,9.7,5.6,"['bulgur', 'spinach', 'cannellini']"


In [42]:
##rename the dataframe id
recipes.index.names = ['recipes_id']
recipes.head(2)

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed
recipes_id,,,,,,,,,,,,,,
0,Dessert Crepes,https://www.allrecipes.com/recipe/19037/desser...,breakfast-and-brunch,4.80,1156,"['4 eggs, lightly beaten ', ' 1\u2009⅓ cups m...",20,8,8 crepes,163.8,17.2,7.7,6.4,"['milk', 'flour', 'sugar']"
1,Chicken Parmesan,https://www.allrecipes.com/recipe/223042/chick...,world-cuisine,4.83,4245,"['4 skinless, boneless chicken breast halves ...",60,4,4 servings,470.8,24.8,24.9,42.1,"['chicken breast', 'egg', 'panko bread', 'parm..."


In [43]:
##remove punctuation from parsed_ingredients?
recipes['ingredients_parsed'] = recipes['ingredients_parsed'].str.replace('[^\w\s]','')
recipes.head(2)

/tmp/ipykernel_27474/1542198556.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  recipes['ingredients_parsed'] = recipes['ingredients_parsed'].str.replace('[^\w\s]','')


,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed
recipes_id,,,,,,,,,,,,,,
0,Dessert Crepes,https://www.allrecipes.com/recipe/19037/desser...,breakfast-and-brunch,4.80,1156,"['4 eggs, lightly beaten ', ' 1\u2009⅓ cups m...",20,8,8 crepes,163.8,17.2,7.7,6.4,milk flour sugar
1,Chicken Parmesan,https://www.allrecipes.com/recipe/223042/chick...,world-cuisine,4.83,4245,"['4 skinless, boneless chicken breast halves ...",60,4,4 servings,470.8,24.8,24.9,42.1,chicken breast egg panko bread parmesan cheese...


In [52]:
##for ingredients feature
#define an object for Term frequency-inverse document frequency vectorizer
tfidf = TfidfVectorizer()
tfidf.fit(recipes['ingredients_parsed'])
##fit & tranform data
tfidf_recipes = tfidf.transform(recipes['ingredients_parsed'])
print(tfidf_recipes.shape)
tfidf_recipes.dtype
print(tfidf_recipes)

(19224, 2615)
  (0, 2292)	0.5284839062109208
  (0, 1462)	0.6309099041950023
  (0, 870)	0.5680294478851423
  (1, 2393)	0.23002341512484573
  (1, 1662)	0.5199462696320556
  (1, 1647)	0.39449598005694947
  (1, 742)	0.19366475549318743
  (1, 425)	0.2005113589614988
  (1, 410)	0.54827889536507
  (1, 250)	0.284404083851595
  (1, 244)	0.248820727235273
  (2, 2009)	0.25082688495563565
  (2, 1894)	0.23879179026354988
  (2, 1791)	0.3888150048688435
  (2, 1610)	0.26833227417029404
  (2, 1462)	0.1253889405131509
  (2, 1187)	0.2942960743502115
  (2, 1178)	0.3888150048688435
  (2, 1064)	0.37981233979825135
  (2, 870)	0.11289188864686753
  (2, 587)	0.34481218105858236
  (2, 300)	0.21996096505016413
  (2, 185)	0.28123852009291145
  (3, 2594)	0.3180519089227853
  (3, 2058)	0.42463150900669655
  :	:
  (19218, 1917)	0.2503934176488003
  (19218, 1647)	0.3732531909480908
  (19218, 870)	0.16721650240228683
  (19218, 742)	0.18323631067565957
  (19218, 378)	0.4101786006029754
  (19218, 263)	0.3529233759562378

In [ ]:
##convert the tokenized ingredients into a dataframe
tokenized_ing = pd.DataFrame(tfidf_recipes.toarray(), columns=tfidf.get_feature_names_out(axis=0))
tokenized_ing.sample(2)

In [ ]:
##normalize the numerical column (nutritional values ) in new dataframe << because i dont want to normalize all other features
newrecipes = recipes[['total', 'calories', 'carbohydrates_g', 'fat_g', 'protein_g']]
recipes_norm = pd.DataFrame(normalize(newrecipes, axis=0))
recipes_norm.columns = newrecipes.columns
recipes_norm.index = newrecipes.index
recipes_norm.head(2)

In [53]:
##Concatenate the tokenization ing to the normalized data: <<FAILED!!!! :(
###Make sure that the indices between the two DataFrames are similar!!!!!!!!!
converted_recipes = pd.concat([recipes_norm, tokenized_ing], axis=0)
converted_recipes.head(2)

,total,calories,carbohydrates_g,fat_g,protein_g,abita,about,acai,accent,according,...,yves,zaatar,zapallo,zen,zero,zested,zesty,zing,ziti,zucchini
0,0.000837,0.003888,0.003539,0.003818,0.002726,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.002512,0.011174,0.005103,0.012346,0.017931,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
##define a function for the output(name, url)
def selected_rec(recipe_id):
    recipe_df = recipes.set_index('recipe_id')
    x = "{}  {}".format(recipe_id, recipe_df.at[recipe_id, 'name', 'url'])
    ax.set_title(x)


In [49]:
##recommend top 3 recipes based on how similar they are
##function used was mostly modified from this page: https://www.kaggle.com/code/yyzz1010/content-based-filtering-recipe-recommender
##recipes_norm = normalized nutrition data
##distance method = cosine
##recipe_id: find similar recipes based on the selected recipe
##N: Top N recipe(s)
##return 
# 1) nutrition data of selected recipe and Top N recommendation
# 2) recipe id, recipe name and url of Top N recommendation
def recipe_recommender(distance_method, recipe_id, N):
    start = time()
    
    allRecipes = pd.DataFrame(df_normalized.index)
    allRecipes = allRecipes[allRecipes.recipe_id != recipe_id]
    allRecipes["distance"] = allRecipes["recipe_id"].apply(lambda x: distance_method(df_normalized.loc[recipe_id], df_normalized.loc[x]))
    TopNRecommendation = allRecipes.sort_values(["distance"]).head(N).sort_values(by=['distance', 'recipe_id'])
    # sort by distance then recipe id, the smaller value of recipe id will be picked. 
    
    recipe_df = recipe.set_index('recipe_id')
    recipe_id = [recipe_id]
    recipe_list = []
    image_list = []
    image_path = "/kaggle/input/foodrecsysv1/raw-data-images/raw-data-images/{}.jpg"
    for recipeid in TopNRecommendation.recipe_id:
        recipe_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe(s)
        recipe_list.append("{}  {}".format(recipeid, recipe_df.at[recipeid, 'recipe_name']))
        image_list.append(image_path.format(recipeid))
    
    image_array = []
    for imagepath in image_list:
        img = image.load_img(imagepath)
        img = image.img_to_array(img, dtype='int')
        image_array.append(img)
        
    fig = plt.figure(figsize=(15,15))
    gs1 = gridspec.GridSpec(1, N)
    axs = []
    for x in range(N):
        axs.append(fig.add_subplot(gs1[x]))
        axs[-1].imshow(image_array[x])
    [axi.set_axis_off() for axi in axs]
    for axi, x in zip(axs, recipe_list):
        axi.set_title(x)
    
    end = time()
    running_time = end - start
    print('time cost: %.5f sec' %running_time)
    return df_normalized.loc[recipe_id, :]